# 자연어 생성

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 데이터 블러오기
# Naver movie corpus 불러오기
file = tf.keras.utils.get_file('ratings_train.txt',
  origin='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt',
  extract=True)
df = pd.read_csv(file, sep='\t')

14639104/14628807 [==============================] - 0s 0us/step


In [3]:
df[1000:1007]

,id,document,label
1000,9856453,정말 최고의 명작 성인이 되고 본 이집트의 왕자는 또 다른 감동 그자체네요,1
1001,6961803,이영화만 성공 했어도 스퀘어가 에닉스랑 합병 할일은 없었을텐데..,0
1002,8681713,울컥하는 사회현실 ㅠㅠ,1
1003,5348290,기대를하나도안하면 할일없을때보기좋은영화,0
1004,9340549,소림사 관문 통과하기 진짜 어렵다는거 보여준 영화..극장에서 개봉하는거 반갑다..,1
1005,7357684,시리즈안나오나 ㅠㅠㅠㅠㅠㅠㅠㅠ,1
1006,9303587,끝난다는 사실이 너무 슬퍼요. 가슴이 뻥 뚫려버린것같아..,1


In [4]:
# 데이터 전처리
!pip install konlpy # konlpy 설치시 JDK 1.8, JPype1 라이브러리 버전 확인이 중요!!!

     |████████████████████████████████| 19.4 MB 53.2 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 448 kB 61.0 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [5]:
from konlpy.tag import Okt
okt = Okt()

In [6]:
def word_tokenization(text):
  return [word for word in okt.morphs(text)]

In [7]:
def preprocessing(df):
  df = df.dropna()
  df = df[1000:2000]
  df['document'] = df['document'].str.replace('[^A-Za-z0-9가-힣ㄱ-ㅎㅏ-ㅣ]', '')
  data = df['document'].apply((lambda x: word_tokenization(x)))
  return data

In [8]:
# 텍스트 데이터1000개 전처리 후 불러오기
review = preprocessing(df)
len(review)

1000

In [9]:
print(review[:10])

1000    [정말, 최고, 의, 명작, 성인, 이, 되, 고본, 이집트, 의, 왕자, 는, 또...
1001    [이영화, 만, 성공했어도, 스퀘어, 가, 에, 닉스, 랑, 합병, 할, 일, 은,...
1002                                 [울컥, 하는, 사회, 현실, ㅠㅠ]
1003       [기대, 를, 하나, 도안, 하, 면, 할, 일, 없을, 때, 보기, 좋은, 영화]
1004    [소림사, 관문, 통과, 하, 기, 진짜, 어렵다는거, 보여준, 영화, 극장, 에서...
1005                              [시리즈, 안, 나오나, ㅠㅠㅠㅠㅠㅠㅠㅠ]
1006        [끝난다는, 사실, 이, 너무, 슬퍼요, 가슴, 이, 뻥, 뚫려, 버린것, 같아]
1007                                             [펑점, 조절]
1008                            [와이, 건, 진짜, 으리, 으리, 한, 데]
1009                                [손발, 이, 오, 그라드, 네, 요]
Name: document, dtype: object


In [10]:
# 토큰화 및 패딩
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
def get_tokens(review):
  tokenizer.fit_on_texts(review)
  total_words = len(tokenizer.word_index)+1
  tokenized_sentences = tokenizer.texts_to_sequences(review)

  input_sequences = []
  for token in tokenized_sentences:
    for t in range(1, len(token)):
      n_gram_sequence = token[:t+1]
      input_sequences.append(n_gram_sequence)
  return input_sequences, total_words
input_sequences, total_words = get_tokens(review)
input_sequences[31:40]

[[730, 114, 515, 124],
 [730, 114, 515, 124, 96],
 [389, 8],
 [389, 8, 215],
 [389, 8, 215, 150],
 [389, 8, 215, 150, 17],
 [389, 8, 215, 150, 17, 61],
 [389, 8, 215, 150, 17, 61, 136],
 [389, 8, 215, 150, 17, 61, 136, 105]]

In [11]:
# 단어 사전
print('감동 ==>> ', tokenizer.word_index['감동'])
print('영화 ==>> ', tokenizer.word_index['영화'])
print('코밍 ==>> ', tokenizer.word_index['코믹'])

감동 ==>>  46
영화 ==>>  2
코밍 ==>>  392


In [12]:
# 패딩
max_len = max([len(word) for word in input_sequences])
print('max_len:', max_len)

max_len: 63


In [13]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_len, padding='pre'))

In [14]:
# 입력텍스트와 타겟
from tensorflow.keras.utils import to_categorical
x = input_sequences[:,:-1]
y = to_categorical(input_sequences[:, -1], num_classes=total_words)

In [ ]:
# 모델
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout

embedding_dim = 256
model = Sequential([
                    Embedding(input_dim=total_words,
                              output_dim=embedding_dim,
                              input_length=max_len-1),
                    Bidirectional(LSTM(units=256)),
                    Dense(units=total_words, activation='softmax')
                    ])
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(x, y, epochs=20, verbose=2)

Epoch 1/20
354/354 - 220s - loss: 7.8739 - accuracy: 0.0202
Epoch 2/20
354/354 - 215s - loss: 7.2584 - accuracy: 0.0249
Epoch 3/20
354/354 - 217s - loss: 6.9088 - accuracy: 0.0327
Epoch 4/20
354/354 - 216s - loss: 6.3978 - accuracy: 0.0464
Epoch 5/20
354/354 - 221s - loss: 5.6528 - accuracy: 0.0728
Epoch 6/20
354/354 - 217s - loss: 4.7403 - accuracy: 0.1377
Epoch 7/20


In [ ]:
# 문장생성함수 (시작 텍스트, 생성 단어 수)
def text_generation(sos, count):
  for _ in range(1, count):
    token_list = tokenizer.texts_to_sequences([sos])[0]
    token_list = pad_sequences([token_list], maxlen=max_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=1) # 예측값의 최대 값 인텍스
    for word, idx in tokenizer.word_index.items():
      if idx == predicted:
        output = word
        break
    sos += ' ' + output
  return sos

In [ ]:
text_generation('연애 하면서', 12)

In [ ]:
text_generation('꿀잼', 12)

In [ ]:
text_generation('최고의 영화', 12)